In [1]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, isdir, join
import ast
import sys
import time
import datetime

#vader
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import re
import string
nltk.download('stopwords')
from nltk.corpus import stopwords

#Display Setting
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)

#detect language
from textblob import TextBlob  

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Ace\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ace\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
## Useful functions
def seconds_time(s):
    seconds_set = " "+str(s%60)+"sec" if s%60!=0 else ""
    s = s//60
    
    mins_set = " "+str(s%60)+"min" if s%60!=0 else ""
    s = s//60
    
    hrs_set = " "+str(s%24)+"hr" if s%24!=0 else ""
    s = s//24
    
    s = str(s)+"days" if s!=0 else ""
    
    return s+hrs_set+mins_set+seconds_set

In [3]:
start_time_df = datetime.datetime.now()

from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

sc = SparkContext("local[2]")
spark = SparkSession.builder.master("local[2]").getOrCreate()

#directory of the unzipped data profile
df = spark.read.json(r".\*\*\submissions\*.json")

stop_time_df = datetime.datetime.now()
print("Data done extract in "+seconds_time((stop_time_df-start_time_df).seconds)+" with count of "+str(df.count()))

Data done extract in  4min 13sec with count of 296357


# Content of data
- author: author name
- author_fullname: author ID
- created_utc: Time to be converted to normal datetime
- full_link: link of submission
- id: id of submission. To be used to link from comments
- num_comments: number of comments
- num_crossposts: number of cross posts
- score: Not sure but keeping just in case its some special score
- subreddit: subreddit name
- subreddit_id: subreddit id
- subreddit_subscribers: number of subscribers in the subreddit
- subreddit_type: Type of subreddit, whether public or private
- title: title of submission

In [5]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [6]:
df_set = df.withColumn("created_utc", F.col("created_utc")/1000)
df_set = df_set.withColumn("created_utc", F.from_unixtime("created_utc"))
df_set = df_set.orderBy(F.col("created_utc").asc())
df_set = df_set.dropDuplicates()
df_set = df_set.orderBy(F.col("created_utc").asc())

In [7]:
start_time = datetime.datetime.now()
display(df_set.show())
stop_time = datetime.datetime.now()
print("Data done extract in "+seconds_time((stop_time-start_time).seconds))

+-------------------+---------------+-------------------+-------------------+--------------------+------+------------+--------------+-----+--------------------+--------------+------------+----------------+---------------------+--------------+--------------------+
|             author|author_fullname|author_fullname_lvl|        created_utc|           full_link|    id|num_comments|num_crossposts|score|            selftext|     subreddit|subreddit_id|subreddit_id_lvl|subreddit_subscribers|subreddit_type|               title|
+-------------------+---------------+-------------------+-------------------+--------------------+------+------------+--------------+-----+--------------------+--------------+------------+----------------+---------------------+--------------+--------------------+
|   CautiousInvestor|       2gcxulwq|                 t2|2019-01-01 08:01:42|https://www.reddi...|ab8cj7|         412|             0|    1|                    |     investing|       2qhhq|              t5|   

None

Data done extract in  2min 50sec


In [9]:
df_set.toPandas().to_csv("submissions_full.csv", index=False)